In [1]:
import openpyxl
import xlwings as xw
import pandas as pd
import shutil
import io
import os
import pandas as pd
import time

In [2]:
user_profile = os.getenv('USERPROFILE') ## Retrieving user profile directory from 'USERPROFILE' environment variable.

In [3]:
# Constructing the base directory path using the user profile directory.
base_dir = os.path.join(
    user_profile, 
    'California Department of Transportation',
    'DOT HQ PMP Cal B C Update - General',
    'Testbed'
)

In [4]:
# CalB/C Available Models 
at_model = os.path.join(base_dir, 'Input', 'CalBC Models', 'AT_model.xlsm')
sketch_model = os.path.join(base_dir, 'Input', 'CalBC Models', 'Sketch_model.xlsm')
if_model = os.path.join(base_dir, 'Input', 'CalBC Models', 'Intermodal_freight_model.xlsm')
pr_model = os.path.join(base_dir, 'Input', 'CalBC Models', 'Park_&_ride_model.xlsm')
corridor_model = os.path.join(base_dir, 'Input', 'CalBC Models', 'Corridor_model.xlsm')

In [5]:
# Function to get both model and template paths
def get_model_and_template(model_file):
    model_name = os.path.splitext(os.path.basename(model_file))[0]
    template_file = os.path.join(base_dir, 'Input', 'Templates', f"{model_name}_template.xlsx")
    return model_file, template_file

In [6]:
# Constructing file directory for output and template files.
output_dir = os.path.join(base_dir, 'Output')

In [7]:
# Function to identify input cells using cell background 
def identify_green_cells(main_file, sheet_names):
    """Identify green-highlighted cells in the specified sheet names and return their cell addresses."""
    green_cells_addresses = {}
    
    with xw.App(visible=False) as app:
        main_wb = app.books.open(main_file)
        print(main_wb.sheet_names)
        
        # Loop through the specified sheet names
        for sheet_name in sheet_names:
            if sheet_name in main_wb.sheet_names:
                sheet = main_wb.sheets[sheet_name]
                green_cells_addresses[sheet_name] = []
                
                # Loop through all the rows in the used range
                for row in sheet.used_range.rows:  
                    for cell in row:  
                        # Check if the cell is highlighted in green (color: #CCFFCC)
                        if cell.color == (204, 255, 204):  

                            # Add the cell's address to the list
                            green_cells_addresses[sheet_name].append(cell.address)
        
        main_wb.close()
    
    return green_cells_addresses



In [ ]:
# Specify which model to use
main_file, template_file = get_model_and_template(corridor_model)

# Specify which sheets have inputs for the given model 
sheet_names = ["1) Project Information"] 

In [9]:
#Run the function 
green_cells_addresses = identify_green_cells(main_file, sheet_names)
print(green_cells_addresses)


['Title', 'Instructions', '1) Project Information', '2) Model Inputs', '3) Results', 'Travel Time', 'Consumer Surplus', 'Reliability', 'Vehicle Operating Costs', 'Accident Costs', 'Emissions', 'Final Calculations', 'PARAMETERS']
{'2) Model Inputs': ['$F$11', '$G$11', '$H$11', '$I$11', '$J$11', '$K$11', '$L$11', '$M$11', '$N$11', '$V$11', '$W$11', '$X$11', '$Y$11', '$AA$11', '$AB$11', '$AC$11', '$AD$11', '$AL$11', '$AM$11', '$AN$11', '$AO$11', '$AP$11', '$AQ$11', '$AR$11', '$AS$11', '$BQ$11', '$BR$11', '$BS$11', '$BT$11', '$BU$11', '$BV$11', '$BW$11', '$BX$11', '$BY$11', '$BZ$11', '$CA$11', '$CB$11', '$CD$11', '$CE$11', '$CG$11', '$CH$11', '$CK$11', '$CL$11', '$CN$11', '$CO$11', '$CX$11', '$DH$11', '$F$29', '$G$29', '$H$29', '$I$29', '$J$29', '$K$29', '$V$29', '$W$29', '$X$29', '$Y$29', '$Z$29', '$AA$29', '$AM$29', '$AN$29', '$AO$29', '$AP$29', '$BA$29', '$BP$29', '$BQ$29', '$BR$29', '$CD$29', '$CE$29', '$CF$29', '$F$35', '$G$35', '$H$35', '$I$35', '$J$35', '$K$35', '$V$35', '$W$35', '$

In [10]:
def populate_green_cells(main_file, template_file, green_cells_addresses, output_dir):
    """Populate green-highlighted cells in the main file from the template file for each version."""
    with xw.App(visible=False) as app:
        # Open the main file and the template file
        main_wb = app.books.open(main_file)
        template_wb = app.books.open(template_file)
        
        # Loop through each version
        for version in ['1.0', '2.0']:
            print(f"Processing version {version}")
            
            # Loop through the sheets in the main file and the green_cells_addresses dictionary
            for sheet_name, green_cells in green_cells_addresses.items():
                if sheet_name in main_wb.sheet_names:
                    sheet = main_wb.sheets[sheet_name]
                    
                    # Check for the versioned sheet name in the template file
                    template_sheet_name = f"{sheet_name}_{version}"  
                    
                    
                    if template_sheet_name in template_wb.sheet_names:
                        template_sheet = template_wb.sheets[template_sheet_name]

                        
                        # Loop through the green-highlighted cells and copy data from the template
                        for address in green_cells:
                            cell = sheet[address]
                            template_cell = template_sheet[address]

                            # Copy the data from the template to the main file
                            cell.value = template_cell.value
                    
            # Recalculate all formulas in the main file
            main_wb.app.calculate()

            # Save the updated main file with the version-specific suffix
            output_filename = f"{os.path.splitext(os.path.basename(main_file))[0]}_v{version}.xlsx"
            output_filepath = os.path.join(output_dir, output_filename)
            main_wb.save(output_filepath)
            print(f"Saved {output_filepath}")
        
        main_wb.close()
        template_wb.close()

In [11]:
# Running the function 
populate_green_cells(main_file, template_file, green_cells_addresses, output_dir)

Processing version 1.0
Saved C:\Users\s159357\California Department of Transportation\DOT HQ PMP Cal B C Update - General\Testbed\Output\Corridor_model_v1.0.xlsx
Processing version 2.0
Saved C:\Users\s159357\California Department of Transportation\DOT HQ PMP Cal B C Update - General\Testbed\Output\Corridor_model_v2.0.xlsx
